# **3 - CREACIÓN MODELO + ENTRENAMIENTO**

 <a name='ind'/>

## <ins>Indice</ins>:

#### 0. [Importaciones](#imp)
#### 1. [Análisis de variables](#ana)
#### 2. [Creación y entrenamiento del modelo](#model)
#### 4. [Evaluación del modelo](#eval)
***

En este notebook usaremos la los datos ya transformados (operación realizada en el [notebook anterior](EDA.ipynb)) para aplicar un modelo de Machine Learning. Optaremos por un modelo de random forest, y escogeremos sus parámetros usando una búsqueda aleatoria de distintas opciones, minimzando el error cuadrático medio.

Analizaremos los datos y evaluaremos la colinealidad de sus columnas, y con ello decidiremos con que variables entrenamos al modelo

### Importaciones
***

In [166]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
pd.set_option('display.max_columns', None) 

import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

<a name='trans'/>

###### ⬆️ [Indice](#ind)

### Análisis de variables
***

Cargamos los datos ya transformados

In [60]:
# Datos entrenamiento
X_train = pd.read_csv('../data/transform/X_train_transform.csv')
X_train_no_cluster = X_train.drop(columns=['4_cluster','5_cluster'])
X_train_5c =  X_train.drop(columns=['4_cluster'])
X_train_4c =  X_train.drop(columns=['5_cluster'])

y_train = pd.read_csv('../data/transform/y_train.csv')

# Datos de testeo
X_test = pd.read_csv('../data/transform/X_test_transform.csv')
X_test_no_cluster = X_test.drop(columns=['4_cluster','5_cluster'])
X_test_5c =  X_test.drop(columns=['4_cluster'])
X_test_4c =  X_test.drop(columns=['5_cluster'])

Evaluamos colinealidad en los datos

In [150]:
def corr_plot(X,y):
    # Creamos matriz de correlación
    corr = pd.concat([X,y],axis=1).corr()

    # Creamos máscara
    mask = np.triu(np.ones_like(corr,dtype=bool))
    data = corr[~pd.DataFrame(mask,index=corr.index,columns=corr.columns)]

    # Incializamos figura
    return px.imshow(abs(data).round(4),text_auto=True,color_continuous_scale='reds').update_layout(width=1000,height=800)

In [152]:
corr_plot(X_train,y_train)

Vemos que los datos no guardan una colinealidad elevada, salvo en valores de version del sistema operativo. Algo esperado, ya que en el proceso de limpieza y transformación, hemos completado muchos de estos valores son iguales.

Sabiendo esto, haremos varios entrenamientos del modelo, incluyendo o no estas columnas con alta colinealidad

<a name='model'/>

###### ⬆️ [Indice](#ind)

### Creación y entrenamiento del modelo
***

Entrenaremos diversos modelos de Randon Forest, y buscaremos los mejores parámetros de ajuste usando un RandomizedSearch.

Hemos optado por este modelo, ya que por lo general se ajustan mejor que un modelo de regresión lineal, teniendo la pega de que el modelo tenga un alto 'overfit' con los datos de entrenamiento y no prediga correctamente nuevos datos

Definimos una función para hacer todas las combinaciones posibles de parámatros y nos devuelva el modelo que minimice el error cuadrático medio

In [142]:
def train(X,y):  
    n_estimators=[int(x) for x in np.linspace(200, 800, 10)]  

    max_features=['auto']   

    min_samples_split=[2, 3, 4, 5]
    
    bootstrap=[True, False]  

    params={'n_estimators': n_estimators,
            'max_features': max_features,
            'min_samples_split':min_samples_split,
            'bootstrap': bootstrap}

    rfr = RFR()

    rf_random = RandomizedSearchCV(estimator=rfr,
                                   param_distributions=params,
                                   n_iter=80,
                                   cv=5,
                                   n_jobs=-1,
                                   verbose=0,
                                   scoring='neg_root_mean_squared_error')

    return rf_random.fit(X,y)

A la hora de entrenar el modelo vamos a tener en cuenta los siguientes factores:
- Presencia de outliers: entrenar al modelo con outliers, seguramente reduzca el R2 y eleve el error, pero a lo mejor hace que prediga mejor nuevos valores no conocidos, ya que será más general
- Clusters: En el proceso de transformación hemos etiquetado cada registro con un clústrer, esto puede hacer que el modelo mejore, ya que tiene información adicional, o bien puede empeorarlo ya que podemos dara dos veces la misma información al modelo y crear ruido
- Colinealidad: Entrenar al modelo incluyendo características que guarden alta correlación, puede generar un modelo con mucho ruido y que prediga correctamtne

Teneiendo esto en cuenta, entrenaremos a los modelos con las siguientes combinaciones y analizaremos posteriormente los resultados:
- Con y sin outliers
- sin cluster con clusters (4 y 5)
- Quitando columna version y no version

In [155]:
# Cargamos los indices de los outliers detectados en el EDA
outliers = pd.read_csv('../data/transform/outliers_index.csv')

In [ ]:
%%time

# Creamos una lista donde almacenar todos los modelos que se entrene
modelos = []

for out in range(2):
    for cluster in [0,4,5]:
        for version in range(2):
            # Generamos los datasets de pruebas según la combinatoria previamente definida
            if not out:
                index = outliers.iloc[:,0]
            else:
                index = []
            if cluster == 0:
                X = X_train_no_cluster.drop(index=index)
                X_t = X_test_no_cluster
            elif cluster == 4:
                X = X_train_4c.drop(index=index)
                X_t = X_test_4c
            else:
                X = X_train_5c.drop(index=index)
                X_t = X_test_5c
            if not version:
                X = X.drop(columns=['Operating_System_Version_10 S',
                                    'Operating_System_Version_7',
                                    'Operating_System_Version_Android',
                                    'Operating_System_Version_Chrome OS',
                                    'Operating_System_Version_No OS',
                                    'Operating_System_Version_Ubuntu',
                                    'Operating_System_Version_X'                                   
                                   ])
                X_t = X_t.drop(columns=['Operating_System_Version_10 S',
                                        'Operating_System_Version_7',
                                        'Operating_System_Version_Android',
                                        'Operating_System_Version_Chrome OS',
                                        'Operating_System_Version_No OS',
                                        'Operating_System_Version_Ubuntu',
                                        'Operating_System_Version_X'                                   
                                       ])
            
            modelo = train(X,y_train.drop(index=index))    # Entrenamos el modelo 
            y_hat = modelo.best_estimator_.predict(X)      # Calculamos la mejor predicción para el train
            
            # Calculamos métricas de evaluación
            score = r2(y_hat,y_train.drop(index=index))
            mse_value = mse(y_hat,y_train.drop(index=index),squared=False)
            
            # Guardamos el modelo y la información relevante del entrenamiento
            
            modelos.append({'outliers':out,
                            'cluster':cluster,
                            'version':version,
                            'modelo': modelo,
                            'X':X,
                            'y':y_train.drop(index=index),
                            'X_test':X_t,
                            'R2':score,
                            'mse':mse_value,
                            'y_hat':y_hat
                           })

<a name='eval'/>

###### ⬆️ [Indice](#ind)

### Evaluación del modelo
***

Tenemos 12 modelos entrenados, tenemos que evaluar el que mejor se ajusta

In [80]:
# Creamos un dataframe con la información y generamos predicciones

df_models = pd.DataFrame(columns=['Outliers','Cluster','version','R2','mse'])

for index,m in enumerate(modelos):
    df_models.loc[index]=[m['outliers'],m['cluster'],m['version'],m['R2'],m['mse']]
    
    # Generamos y guardamos predicción
    
    muestra = pd.read_csv('../data/muestras/muestra.csv')
    y_hat_test = m['modelo'].best_estimator_.predict(m['X_test'])
    muestra['Price'] = y_hat_test
    muestra.to_csv(f'../data/muestras/muestra{m["outliers"]}_{m["cluster"]}_{m["version"]}.csv',index=False)   

Vemos y grafícamos las métricas de cada modelo

In [81]:
df_models

,Outliers,Cluster,version,R2,mse
0,0.0,0.0,0.0,0.976735,755969.018577
1,0.0,0.0,1.0,0.977759,739773.487695
2,0.0,4.0,0.0,0.977526,742915.688039
3,0.0,4.0,1.0,0.977768,739730.901528
4,0.0,5.0,0.0,0.968001,880399.169371
5,0.0,5.0,1.0,0.973802,799216.851028
6,1.0,0.0,0.0,0.979321,864220.493039
7,1.0,0.0,1.0,0.979753,855306.198935
8,1.0,4.0,0.0,0.979413,862522.681484
9,1.0,4.0,1.0,0.980012,851279.362074


Observamos que obtenemos mejores datos de R2 si tenemos en cuenta los outliers (color amarillo), y tenemos mejor respuesta contando con todas las columnas (cuadrados) que quitando columnas con colinealidad

Observamos un comportamiento parecido con el MSE, aunque en este caso el error es menor si no tenemos en cuenta los outliers, algo esperado

In [197]:
fig = make_subplots(rows=1,cols=2, subplot_titles=['R2','MSE'] )
fig.add_trace(go.Scatter(x=df_models.Cluster,y=df_models.R2,marker=dict(size=12,color=df_models.Outliers,symbol=df_models['version']),mode='markers'),row=1,col=1)
fig.add_trace(go.Scatter(x=df_models.Cluster,y=df_models.mse,marker=dict(size=12,color=df_models.Outliers,symbol=df_models['version']),mode='markers'),row=1,col=2)
fig.update_layout(showlegend=False)
fig.show()

Se han subido varias muestras de las obtenidas anteriormente a la competición, y los errores obtendios son 3 veces superiores a los obtenidos en las pruebas. Esto nos da información de que el modelo tiene un elevado 'Overfit'

Para encontrar modelos menos ajustados, y que aboroban mejor la generalización de los datos, vamos a obtener los modelos que han quedado en la mitad en la clasificación de cada entrennamiento.

In [198]:
# Creamos dataframe
df_models_m = pd.DataFrame(columns=['Outliers','Cluster','version','R2','mse'])

for index, m in enumerate(modelos):
    # Obtenemos los modelos que han quedado en posicion 40 
    index_m = m['modelo'].cv_results_['rank_test_score'].tolist().index(40)
    params = (m['modelo'].cv_results_['params'][index_m])
    
    # Obtenemos los datos de entreno y testeo
    X = m['X']
    y = m['y']
    X_t = m['X_test']
    rfr = RFR(**params)
    
    # Entrenamos el modelo y hacemos predicciones
    rfr.fit(X,y)
    y_hat = rfr.predict(X)
    y_hat_test = rfr.predict(X_t)
    
    # Guardamos datos
    muestra = pd.read_csv('../data/muestras/muestra.csv')
    muestra['Price'] = y_hat_test
    muestra.to_csv(f'../data/muestras/muestra_m{index}.csv',index=False)
    df_models_m.loc[index] = [m['outliers'],m['cluster'],m['version'],r2(y_hat,y),mse(y_hat,y,squared=False)]

df_models_m.sort_values('R2')

,Outliers,Cluster,version,R2,mse
0,0.0,0.0,0.0,0.960855,9.684745e+05
5,0.0,5.0,1.0,0.961179,9.639128e+05
4,0.0,5.0,0.0,0.961337,9.622648e+05
3,0.0,4.0,1.0,0.961746,9.573596e+05
1,0.0,0.0,1.0,0.962108,9.539985e+05
11,1.0,5.0,1.0,0.966068,1.098794e+06
9,1.0,4.0,1.0,0.966330,1.090149e+06
6,1.0,0.0,0.0,0.966421,1.087644e+06
8,1.0,4.0,0.0,0.966812,1.084203e+06
2,0.0,4.0,0.0,0.967431,8.866682e+05


Como esperábamos tenemos modelos menos ajustados. Estos modelos seguramente se ajusten más a datos nuevos, por lo que subiremos estos datos a la competición

In [202]:
px.scatter(x= df_models_m.R2,y = df_models_m.mse,color=df_models_m.Outliers.astype(str),symbol=df_models.version,size=df_models_m.Cluster).update_traces(marker=dict(size=12))